In [88]:
import os
from glob import glob
import yaml

from jinja2 import Template

import nbformat as nbf
import papermill as pm

In [99]:
def modify_markdown_header(input_path, output_path, **kwargs):
    notebook = nbf.read(input_path, nbf.NO_CONVERT)
    cells = []
    for cell in notebook.cells:
        if cell.cell_type == 'markdown':
            print(cell['source'])
            new_value = Template(cell['source']).render(**kwargs)
            cell['source'] = new_value
            print(new_value)
            print()
        cells.append(cell)
    notebook.cells = cells
    nbf.write(notebook, output_path, version=nbf.NO_CONVERT)

In [100]:
control = yaml.safe_load(
    """
output_dir: ./output
kernel_name: metabolic
notebooks:
    hello-world:
      parameter_groups:
        a=10_b=1:
          a: 10
          b: 1
    notebook2:
      kernel_name: sno
      parameter_groups:
        temperature:
          variable_id: TEMP
        salinity:
          variable_id: SALT
        oxygen:
          variable_id: O2
    """)
control

{'output_dir': './output',
 'kernel_name': 'metabolic',
 'notebooks': {'hello-world': {'parameter_groups': {'a=10_b=1': {'a': 10,
     'b': 1}}},
  'notebook2': {'kernel_name': 'sno',
   'parameter_groups': {'temperature': {'variable_id': 'TEMP'},
    'salinity': {'variable_id': 'SALT'},
    'oxygen': {'variable_id': 'O2'}}}}}

In [101]:
output_dir = "." if 'output_dir' not in control else control["output_dir"]
os.makedirs(output_dir, exist_ok=True)

if 'kernel_name' in control:
    for d in control['notebooks'].values():
        if 'kernel_name' not in d:
            d['kernel_name'] = control['kernel_name']

In [102]:
files = glob(f"{output_dir}/*.ipynb")
for f in files:
    os.remove(f)

In [104]:

for nb, info in control['notebooks'].items():
    for key, parms in info['parameter_groups'].items():
        
        input_path = f'{nb}.ipynb'        
        output_path = f'{output_dir}/{nb}-{key}.ipynb'
        
        o = pm.execute_notebook(
            input_path=input_path,
            output_path=output_path,
            kernel_name=info['kernel_name'],
            parameters=parms,
        )
        modify_markdown_header(output_path, output_path, **parms)

Executing:   0%|          | 0/8 [00:00<?, ?cell/s]

# Title

This notebooks explores the implications of of a = {{a}} and b = {{b}}
# Title

This notebooks explores the implications of of a = 10 and b = 1

Now compute a function using parameters a = {{a}} and b = {{b}}.
Now compute a function using parameters a = 10 and b = 1.



Executing:   0%|          | 0/4 [00:00<?, ?cell/s]

# Another notebook doing some computation
This notebook operates on `{{variable_id}}`.
# Another notebook doing some computation
This notebook operates on `TEMP`.



Executing:   0%|          | 0/4 [00:00<?, ?cell/s]

# Another notebook doing some computation
This notebook operates on `{{variable_id}}`.
# Another notebook doing some computation
This notebook operates on `SALT`.



Executing:   0%|          | 0/4 [00:00<?, ?cell/s]

# Another notebook doing some computation
This notebook operates on `{{variable_id}}`.
# Another notebook doing some computation
This notebook operates on `O2`.

